# Исследование надежности заемщиков.

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Описание данных:**
- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита

## Шаг 1. Обзор данных

In [1]:
#импорт библиотеки Pandas
import pandas as pd
import numpy as np

In [2]:
#чтение файла с данными
import os
pth1 = '/datasets/data.csv'
pth2 = 'E:\Файлы Галя\Yandex_practicum\Datasets\data.csv'
if os.path.exists(pth1):
    df = pd.read_csv(pth1)
elif os.path.exists(pth2):
    df = pd.read_csv(pth2)
else:
    print("Проверьте правильность пути к датасету")

Выведем первые 10 строк таблицы, чтобы частично ознакомиться с данными и наглядно представлять, с какими данными имеем дело

In [3]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


На первый взгляд все понятно, но сразу видим, что в столбце **days_employed** странные отрицательные значения, а в столбце **education** видим наличие дубликатов, но с этим разберемся позже.

Выведем общую информацию о таблице с данными:

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Из полученной информации видно, что имеются пропуски в столбцах **days_employed** и **total_income**, при этом они имеют одинаковое кол-во non-null, здесь скорее всего есть взаимосвязь. Возможно, это те, у кого еще нет стажа работы, и они еще не зарабатывают.

Все названия столбцов в нормальном формате, значит, их обрабатывать не нужно. 

Проверим количество пропусков в столбцах **days_employed** и **total_income**:

In [5]:
print(df['total_income'].isna().sum())
print(df['days_employed'].isna().sum())

2174
2174


Nan - в обоих стобцах (days_employed, total_income) по 2174 значения

Найдем все строки с пропусками в столбце **total_income** и посмотрим первые пятнадцать, чтобы определить, есть ли зависимость пропущенных значений столбца **days_employed** с пропущенными значениями стобца **total_income**:

In [6]:
df[df['total_income'].isna()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Из представленных данных видно, что во всех строках, где пропущены значения в **days_employed**, так же пропущены значения в **total_income**, что указывает на связь данных в этих столбцах

## Шаг 2. Проверка данных на аномалии и исправления.

Проверим  и исправим данные в каждом столбце отдельно

### 1. Столбец *children*

Узнаем количество уникальных значений в столбце **children**:

In [7]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Больше всего бездетных, затем люди, имеющие 1 и 2 ребенка. Выбивающиеся значения "20" и "-1" - очень странно, что таких значений достаточное клоличество, если бы это были бы просто опечатки, то вряд-ли бы их было больше пары штук, а здесь 76 и 47 соответственно. Нужно проверить в каких строках они встречаются, и если на результат не будут оказывать влияния, то можно их удалить (их незначительное количество - 123 из 21525 - меньше 0,6%)

### Удаление строк с аномальными значениями

Посмотрим, что в строках со значением "-1"

In [8]:
df[df['children']==-1].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
291,-1,-4417.703588,46,среднее,1,гражданский брак,1,F,сотрудник,0,102816.346412,профильное образование
705,-1,-902.084528,50,среднее,1,женат / замужем,0,F,госслужащий,0,137882.899271,приобретение автомобиля
742,-1,-3174.456205,57,среднее,1,женат / замужем,0,F,сотрудник,0,64268.044444,дополнительное образование
800,-1,349987.852217,54,среднее,1,Не женат / не замужем,4,F,пенсионер,0,86293.724153,дополнительное образование
941,-1,NaN,57,Среднее,1,женат / замужем,0,F,пенсионер,0,NaN,на покупку своего автомобиля
1363,-1,-1195.264956,55,СРЕДНЕЕ,1,женат / замужем,0,F,компаньон,0,69550.699692,профильное образование
1929,-1,-1461.303336,38,среднее,1,Не женат / не замужем,4,M,сотрудник,0,109121.569013,покупка жилья
2073,-1,-2539.761232,42,среднее,1,в разводе,3,F,компаньон,0,162638.609373,покупка жилья
3814,-1,-3045.290443,26,Среднее,1,гражданский брак,1,F,госслужащий,0,131892.785435,на проведение свадьбы
4201,-1,-901.101738,41,среднее,1,женат / замужем,0,F,госслужащий,0,226375.766751,операции со своей недвижимостью


Посмотрим, что в строках со значением "20"

In [9]:
df[df['children']==20].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,-855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,-3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,-2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,-2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
3302,20,NaN,35,среднее,1,Не женат / не замужем,4,F,госслужащий,0,NaN,профильное образование
3396,20,NaN,56,высшее,0,женат / замужем,0,F,компаньон,0,NaN,высшее образование
3671,20,-913.161503,23,среднее,1,Не женат / не замужем,4,F,сотрудник,0,101255.492076,на покупку подержанного автомобиля
3697,20,-2907.910616,40,среднее,1,гражданский брак,1,M,сотрудник,0,115380.694664,на покупку подержанного автомобиля
3735,20,-805.044438,26,высшее,0,Не женат / не замужем,4,M,сотрудник,0,137200.646181,ремонт жилью


Зависимости с данными в других столбцах не наблюдается, поэтому можем удалить эти строки

In [10]:
#индексы строк, в которых значение -1
one = df[df['children']==-1].index 

# индексы строк, в которых значение 20
twenty = df[df['children']==20].index  

#удаление строк, в которых значение -1
drop_count_one = 0
try:
    for drop_str in one:
        df = df.drop(drop_str)
        drop_count_one+=1
except:
    print("Не удалось удалить строку: ")
print("Удалено строк: ", drop_count_one)

#удаление строк, в которых значение 20
drop_count_twenty = 0
try:
    for drop_str in twenty:
        df = df.drop(drop_str)
        drop_count_twenty+=1
except:
    print("Не удалось удалить строку: ")    
print("Удалено строк: ", drop_count_twenty)

print("Всего осталось строк:", len(df) - drop_count_one - drop_count_twenty)

Удалено строк:  47
Удалено строк:  76
Всего осталось строк: 21279


In [11]:
#проверка
df[df['children']==20]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Таблица пустая, значит необходимые строки удалились

In [12]:
#проверка
df[df['children']==-1]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


Таблица пустая, значит необходимые строки удалились

In [13]:
#исправим индексы после удаления строк
df = df.reset_index(drop=True)

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21402 entries, 0 to 21401
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21402 non-null  int64  
 1   days_employed     19240 non-null  float64
 2   dob_years         21402 non-null  int64  
 3   education         21402 non-null  object 
 4   education_id      21402 non-null  int64  
 5   family_status     21402 non-null  object 
 6   family_status_id  21402 non-null  int64  
 7   gender            21402 non-null  object 
 8   income_type       21402 non-null  object 
 9   debt              21402 non-null  int64  
 10  total_income      19240 non-null  float64
 11  purpose           21402 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Теперь в таблице 21402 строки, было удалено 123 строки

### Столбец *days_employed*

### Изменение некорректных данных

В столбце **days_employed** были обнаружены отрицательные значения, посмотрим, какое общее количество они составляют:

In [15]:
print("Отрицательные значения = ",df[df['days_employed']<0]['days_employed'].count())

Отрицательные значения =  15809


Проведем оценку данных столбца **days_employed** с помощью метода *describe()*, округлив значения до тысячных:

In [16]:
df['days_employed'].describe().round(3)

count     19240.000
mean      63159.821
std      140928.943
min      -18388.950
25%       -2747.236
50%       -1203.934
75%        -289.740
max      401755.400
Name: days_employed, dtype: float64

Присутствуют слишком большие значения (возможно это стаж в часах, а не днях) и много отрицательных (возможно минус это дефис) - нужно исправить эти данные. Также можно уточнить информацию об этих данных у источника, чтобы удостовериться в верности предположений, выдвинутых выше. 
Примем, что предположения верны, поэтому заменим отрицательные значения на положительные.

In [17]:
def negative_to_pozitive(value):
    if value < 0:
        value *= -1
        return value
    else:
        return value
df['days_employed'] = df['days_employed'].apply(negative_to_pozitive)

In [18]:
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Теперь снова проведем оценку данных в столбце **days_employed**:

In [19]:
df['days_employed'].describe().round(3)

count     19240.000
mean      67027.691
std      139130.846
min          24.142
25%         927.984
50%        2195.252
75%        5556.372
max      401755.400
Name: days_employed, dtype: float64

Слишком большие значения максимума и стандартного отклонения, эти значения в **days_employed** игнорировать не можем, так как они могут повлиять на конечный результат. (Вообще тут по-хорошему нужно узнать у источника данных, в каком виде предоставлена информация о стаже (в днях или в часах).) 

Определим максимальный возраст в столбце **dob_years**, чтобы понять, какой максимальный стаж может быть в днях:

In [20]:
df['dob_years'].max()

75

Теперь, зная максимальный возраст, можно определить, начиная с какого значения можно переводить данные в часы.
Так как человек родился в СССР, то (по данным поиска в Яндексе) в то время работать можно было с 16 лет, а в особых случаях, по разрешению специальной комиссии, даже с 14 лет, поэтому примем, что максимальный стаж может быть 60-61 год.

При учете, что в среднем при 8-часовом рабочем дне в году 247 рабочих дней, то нижняя граница для перевода стажа в часы будет примерно 15000 (так как вряд-ли люди моложе 75 лет могли работать 60 лет)

Посмотрим на строки с данными людей, которым 75 лет:

In [21]:
df[df['dob_years']==df['dob_years'].max()]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
8835,0,1678.969771,75,среднее,1,вдовец / вдова,2,F,госслужащий,0,153282.648133,заняться образованием


Оказалось, что 75 лет всего 1 человеку, и стаж не превышает 7 лет, если данные указаны в днях, что вполне вероятно (возможно этот человек работал на себя без оформления ИП, или же, так как это женщина, человек вел домашнее хозяйство).
Даже учитывая этот факт, оставим 15000 как отправную точку для исправления данных в столбце **days_employe**:

Определим количество значений превышающих 15000:

In [22]:
df[df['days_employed']>15000].count()

children            3444
days_employed       3444
dob_years           3444
education           3444
education_id        3444
family_status       3444
family_status_id    3444
gender              3444
income_type         3444
debt                3444
total_income        3444
purpose             3444
dtype: int64

Удалять такое количество данных будет ошибкой, так как эти значения составляют 16% от общего числа.
По этой причине сделаем перевод таких данных в дни, предполагая, что значения выше 15000 будут относиться к значениям в часах, а не днях

In [23]:
df['days_employed'].where(~(df.days_employed >= 15000), other=df.days_employed/24, inplace=True)

In [24]:
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Конечно, остались значения странные, например в пятой строке 14177, то есть при 40 часовой рабочей неделе это примерно 57 лет стажа, но преположим что этот человек работал больше 40 часов в неделю.

Теперь снова проверим информацию по столбцу **days_employed**:

In [25]:
#проверка, все ли значения исправлены
df[df['days_employed']>15000].count()

children            1946
days_employed       1946
dob_years           1946
education           1946
education_id        1946
family_status       1946
family_status_id    1946
gender              1946
income_type         1946
debt                1946
total_income        1946
purpose             1946
dtype: int64

Этот столбец мы подготовили, перейдем к следующему.

### Столбец *dob_years*

In [26]:
print(df['dob_years'].value_counts().sort_index())

0     100
19     14
20     51
21    110
22    183
23    252
24    263
25    356
26    406
27    490
28    501
29    543
30    536
31    556
32    506
33    577
34    597
35    614
36    553
37    531
38    595
39    572
40    603
41    603
42    592
43    510
44    543
45    494
46    469
47    480
48    536
49    505
50    509
51    446
52    483
53    457
54    476
55    441
56    482
57    457
58    461
59    441
60    376
61    353
62    351
63    268
64    263
65    194
66    183
67    167
68     99
69     83
70     65
71     58
72     33
73      8
74      6
75      1
Name: dob_years, dtype: int64


В этом столбце есть значение "0", что странно, так как возраст не может быть 0. Нужно проверить в каких строках это значение  встречается, и если на результат не будет оказывать влияния, то можно их удалить (их незначительное количество: 101 из 21402 - меньше 0,5%)

In [27]:
df[df['dob_years'] == 0]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,14439.234121,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
577,0,16577.356876,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1033,0,1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
...,...,...,...,...,...,...,...,...,...,...,...,...
19716,0,NaN,0,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,жилье
20346,0,14113.952856,0,среднее,1,женат / замужем,0,F,пенсионер,0,259193.920299,покупка своего жилья
20461,0,13822.552977,0,среднее,1,Не женат / не замужем,4,F,пенсионер,0,129788.762899,недвижимость
21060,2,108.967042,0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,строительство жилой недвижимости


Закономерности не прослеживаатся, поэтому удалим эти строки:

### Удаление строк с аномальными значениями

In [28]:
#индексы строк со значением возраста "0"
zeroes = df[df['dob_years']==0].index

#удаляем строки с помощью цикла
drop_zeroes = 0
try:
    for drop_str in zeroes:
        df = df.drop(drop_str)
        drop_zeroes+=1
except:
    print("Не удалось удалить строку")
print("Удалено строк: ", drop_zeroes)

print("Всего осталось строк:", len(df))

Удалено строк:  100
Всего осталось строк: 21302


In [29]:
#исправим индексы после удаления строк
df = df.reset_index(drop=True)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21302 entries, 0 to 21301
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21302 non-null  int64  
 1   days_employed     19150 non-null  float64
 2   dob_years         21302 non-null  int64  
 3   education         21302 non-null  object 
 4   education_id      21302 non-null  int64  
 5   family_status     21302 non-null  object 
 6   family_status_id  21302 non-null  int64  
 7   gender            21302 non-null  object 
 8   income_type       21302 non-null  object 
 9   debt              21302 non-null  int64  
 10  total_income      19150 non-null  float64
 11  purpose           21302 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Теперь в таблице осталось 21302 строки с данными.

### Столбцы *education* и *education_id*

In [31]:
df['education'].value_counts()

среднее                13609
высшее                  4666
СРЕДНЕЕ                  764
Среднее                  700
неоконченное высшее      663
ВЫСШЕЕ                   270
Высшее                   266
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64

Здесь видим дубликаты, которые нужно привести к одному регистру.

In [32]:
df['education_id'].value_counts()

1    15073
0     5202
2      739
3      282
4        6
Name: education_id, dtype: int64

По столбцу **education_id** видно, что существует 5 типов образования. В столбце **education** стоят одни и теже категории, но с разными регистрами, надо эти дубликаты исправить

### Устранение дубликатов

In [33]:
df['education'] = df['education'].str.lower()

Проверим, исправился ли регистр:

In [34]:
df['education'].value_counts()

среднее                15073
высшее                  5202
неоконченное высшее      739
начальное                282
ученая степень             6
Name: education, dtype: int64

Теперь пять типов образования, что соответствует количеству id

In [35]:
df.duplicated().sum()

71

In [36]:
duplicated_df = df[df.duplicated()].head(10)
duplicated_df

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2823,0,NaN,41,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,покупка жилья для семьи
3260,0,NaN,58,среднее,1,гражданский брак,1,F,пенсионер,0,NaN,сыграть свадьбу
4142,1,NaN,34,высшее,0,гражданский брак,1,F,сотрудник,0,NaN,свадьба
4808,0,NaN,60,среднее,1,гражданский брак,1,F,пенсионер,0,NaN,свадьба
5506,0,NaN,58,среднее,1,гражданский брак,1,F,пенсионер,0,NaN,сыграть свадьбу
6258,0,NaN,30,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,строительство жилой недвижимости
7732,0,NaN,57,среднее,1,гражданский брак,1,F,пенсионер,0,NaN,на проведение свадьбы
7845,0,NaN,64,высшее,0,гражданский брак,1,F,пенсионер,0,NaN,на проведение свадьбы
7862,0,NaN,71,среднее,1,гражданский брак,1,F,пенсионер,0,NaN,на проведение свадьбы
8499,0,NaN,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,NaN,дополнительное образование


In [37]:
df = df.drop_duplicates().reset_index(drop=True)

### Столбцы *family_status* и *family_status_id*

In [38]:
df['family_status'].value_counts()


женат / замужем          12213
гражданский брак          4113
Не женат / не замужем     2780
в разводе                 1179
вдовец / вдова             946
Name: family_status, dtype: int64

In [39]:
df['family_status_id'].value_counts()

0    12213
1     4113
4     2780
3     1179
2      946
Name: family_status_id, dtype: int64

Количество совпадает. Оба столбца корректны.

### Столбец *gender*

In [40]:
df['gender'].value_counts()

F      14021
M       7209
XNA        1
Name: gender, dtype: int64

Здесь видим, что женщин больше мужчин, а также появилось какое-то значение с XNA, так как оно одно, то можно его удалить, так как на результат это не повлияет.

In [41]:
df = df.drop(np.where(df['gender'] == 'XNA')[0])

In [42]:
#неверное решение
#df[df['gender']=='XNA']
#df.drop(10595)

In [43]:
#исправим индексы после удаления строк
df = df.reset_index(drop=True)

In [44]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21230 entries, 0 to 21229
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21230 non-null  int64  
 1   days_employed     19149 non-null  float64
 2   dob_years         21230 non-null  int64  
 3   education         21230 non-null  object 
 4   education_id      21230 non-null  int64  
 5   family_status     21230 non-null  object 
 6   family_status_id  21230 non-null  int64  
 7   gender            21230 non-null  object 
 8   income_type       21230 non-null  object 
 9   debt              21230 non-null  int64  
 10  total_income      19149 non-null  float64
 11  purpose           21230 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 1.9+ MB


Теперь осталось 21301 строка с данными

In [45]:
#код ревьюера
df.gender.value_counts()

F    14021
M     7209
Name: gender, dtype: int64

### Столбец *Income_type*

In [46]:
df['income_type'].value_counts()

сотрудник          10961
компаньон           5026
пенсионер           3792
госслужащий         1445
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64

Со столбцом все в порядке, оставим как есть.

### Столбец *debt*

In [47]:
df['debt'].value_counts()

0    19506
1     1724
Name: debt, dtype: int64

Все корректно.

### Столбец *total_income*

In [48]:
df['total_income'].describe()

count    1.914900e+04
mean     1.674919e+05
std      1.032220e+05
min      2.066726e+04
25%      1.030283e+05
50%      1.450179e+05
75%      2.033535e+05
max      2.265604e+06
Name: total_income, dtype: float64

In [49]:
df[df['total_income'].isnull()].head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Похоже что нет закономерности в появлении пропусков с другими столбцами т.к. везде разные значения.

### Заполнение пропусков

In [50]:
df['days_employed'].value_counts()

8437.673028     1
3507.818775     1
14770.850661    1
769.717438      1
3963.590317     1
               ..
16282.044543    1
14219.893165    1
14533.281067    1
2128.237967     1
1984.507589     1
Name: days_employed, Length: 19149, dtype: int64

In [51]:
null = df.loc[(df['days_employed'].isnull()) & (df['total_income'].isnull()),'dob_years'].count()
print("Кол-во значений отсутствующих одновременно в обоих столбцах: ", null)
df.loc[(df['days_employed'].isnull()) & (df['total_income'].isnull())].head(10)

Кол-во значений отсутствующих одновременно в обоих столбцах:  2081


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу
65,0,NaN,21,среднее,1,Не женат / не замужем,4,M,компаньон,0,NaN,операции с коммерческой недвижимостью
67,0,NaN,52,высшее,0,женат / замужем,0,F,пенсионер,0,NaN,покупка жилья для семьи
72,1,NaN,32,высшее,0,женат / замужем,0,M,госслужащий,0,NaN,операции с коммерческой недвижимостью
82,2,NaN,50,высшее,0,женат / замужем,0,F,сотрудник,0,NaN,жилье
83,0,NaN,52,среднее,1,женат / замужем,0,M,сотрудник,0,NaN,жилье


Вывод первых 10 строчек с условием, что столбцы **total_income** и **days_employed** одновременно имеют пропуски показал, что возраст клиентов варьируется от 21 до 65. Значит предположение, что это никогда неработавшие клиенты, не является верным. 

Во всех остальных столбцах тоже стоят разные значения, поэтому будем заполнять **total_income** и **days_employed** медианными значениями. Разница между максимальными и минимальными значениями существенна и среднее значение по этой причине не подходит для заполнения отсутствующих значений, так как это завысит статистику. 

In [52]:
days_employed_median = df['days_employed'].median()
print("Медианное значение days_employed: ", days_employed_median)
total_income_median = df['total_income'].median()
print("Медианное значение total_income: ", total_income_median)

Медианное значение days_employed:  2192.155977727033
Медианное значение total_income:  145017.93753253992


In [53]:
df['days_employed'] = df['days_employed'].fillna(days_employed_median)
df['total_income'] = df['total_income'].fillna(total_income_median)

In [54]:
df.head(15)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,5623.422610,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,14177.753002,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,152.779569,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,6929.865299,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


Проверим, остались ли пропуски

In [55]:
print(df['total_income'].isna().sum())
print(df['days_employed'].isna().sum())

0
0


Все пропущенные значения заменили на медианные.

### Изменение типов данных

In [56]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

В стаже и доходе стояли значения типа float - заменили их на int.

### Столбец *purpose*

In [57]:
df['purpose'].value_counts()

свадьба                                   785
на проведение свадьбы                     759
сыграть свадьбу                           755
операции с недвижимостью                  669
покупка коммерческой недвижимости         655
покупка жилья для сдачи                   647
операции с коммерческой недвижимостью     643
операции с жильем                         641
покупка жилья для семьи                   636
жилье                                     635
покупка жилья                             634
недвижимость                              627
строительство собственной недвижимости    626
операции со своей недвижимостью           623
строительство недвижимости                619
покупка своего жилья                      618
строительство жилой недвижимости          617
покупка недвижимости                      612
ремонт жилью                              602
покупка жилой недвижимости                599
на покупку своего автомобиля              501
заняться высшим образованием      

Столбец корректный, но много совпадающих целей кредита - нужно совпадающие привести к одному названию

### Устранение дубликатов и категоризация данных столбца purpose

In [58]:
# удаление дубликатов
df['purpose'].duplicated().sum()

21192

In [59]:
# функция для категоризации данных
def replace_wrong_values(wrong_values, correct_value):
    for wrong_value in wrong_values:
        df['purpose'] = df['purpose'].replace(wrong_value, correct_value)
duplicates_one = ['на проведение свадьбы', 'сыграть свадьбу']
name_one = 'свадьба'
replace_wrong_values(duplicates_one, name_one)

duplicates_two = ['операции с недвижимостью', 'покупка коммерческой недвижимости', 'покупка жилья для сдачи', 'операции с коммерческой недвижимостью',
                  'операции с жильем', 'покупка жилья для семьи', 'жилье', 'покупка жилья', 'операции со своей недвижимостью',
                  'строительство собственной недвижимости', 'строительство недвижимости', 'строительство жилой недвижимости',
                  'покупка своего жилья', 'покупка недвижимости', 'ремонт жилью', 'покупка жилой недвижимости']
name_two = 'недвижимость'
replace_wrong_values(duplicates_two, name_two)

duplicates_three = ['на покупку своего автомобиля', 'сделка с подержанным автомобилем', 'автомобили', 'на покупку подержанного автомобиля', 
                    'свой автомобиль', 'на покупку автомобиля', 'приобретение автомобиля', 'сделка с автомобилем']
name_three = 'автомобиль'
replace_wrong_values(duplicates_three, name_three)

duplicates_four = ['заняться высшим образованием', 'дополнительное образование', 'высшее образование', 'получение дополнительного образования',
                   'получение образования', 'профильное образование', 'получение высшего образования' , 'заняться образованием']
name_four = 'образование'
replace_wrong_values(duplicates_four, name_four)

In [79]:
# проверим
df['purpose'].value_counts()

недвижимость    10703
автомобиль       4258
образование      3970
свадьба          2299
Name: purpose, dtype: int64

## Шаг 3. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

In [61]:
# таблица с данными об образование
education_table = df[['education_id', 'education']]
education_table.head(15)

,education_id,education
0,0,высшее
1,1,среднее
2,1,среднее
3,1,среднее
4,1,среднее
5,0,высшее
6,0,высшее
7,1,среднее
8,0,высшее
9,1,среднее


In [62]:
# таблица с данными о семейном положении
family_table = df[['family_status_id', 'family_status']]
family_table.head(15)

,family_status_id,family_status
0,0,женат / замужем
1,0,женат / замужем
2,0,женат / замужем
3,0,женат / замужем
4,1,гражданский брак
5,1,гражданский брак
6,0,женат / замужем
7,0,женат / замужем
8,1,гражданский брак
9,0,женат / замужем


In [63]:
# удалим столбцы с образованием и семейным положением, оставим только их id
df = df.drop(columns=['education', 'family_status'])
df.head(10)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437,42,0,0,F,сотрудник,0,253875,недвижимость
1,1,4024,36,1,0,F,сотрудник,0,112080,автомобиль
2,0,5623,33,1,0,M,сотрудник,0,145885,недвижимость
3,3,4124,32,1,0,M,сотрудник,0,267628,образование
4,0,14177,53,1,1,F,пенсионер,0,158616,свадьба
5,0,926,27,0,1,M,компаньон,0,255763,недвижимость
6,0,2879,43,0,0,F,компаньон,0,240525,недвижимость
7,0,152,50,1,0,M,сотрудник,0,135823,образование
8,2,6929,35,0,1,F,сотрудник,0,95856,свадьба
9,0,2188,41,1,0,M,сотрудник,0,144425,недвижимость


In [64]:
#для наглядности выыведем таблицу с id семейного положения
columns = ['family_ststus_id', 'family_status']
data = [['0','женат / замужем '], ['1','гражданский брак'], ['2','Не женат / не замужем'], ['3','в разводе'], ['4','вдовец / вдова']]
purpose_category_table = pd.DataFrame(data=data, columns=columns)
purpose_category_table

,family_ststus_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,Не женат / не замужем
3,3,в разводе
4,4,вдовец / вдова


## Шаг 4. Категоризация дохода.

Оценивать числа, особенно, когда они все разные и большие, достаточно сложно, поэтому лучше категоризировать доходы в зависимости от их размера от D до A.

In [65]:
# функция для категоризации дохода
def total_income_group(total_income):
    if total_income <= 30000:
        return 'E'
    if total_income >= 30001 and total_income <= 50000:
        return 'D'
    if total_income >= 50001 and total_income <= 200000:
        return 'C'
    if total_income >= 200001 and total_income <= 1000000:
        return 'B'
    return 'A'



df['total_income_category'] = df['total_income'].apply(total_income_group)
df.head(15)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category
0,1,8437,42,0,0,F,сотрудник,0,253875,недвижимость,B
1,1,4024,36,1,0,F,сотрудник,0,112080,автомобиль,C
2,0,5623,33,1,0,M,сотрудник,0,145885,недвижимость,C
3,3,4124,32,1,0,M,сотрудник,0,267628,образование,B
4,0,14177,53,1,1,F,пенсионер,0,158616,свадьба,C
5,0,926,27,0,1,M,компаньон,0,255763,недвижимость,B
6,0,2879,43,0,0,F,компаньон,0,240525,недвижимость,B
7,0,152,50,1,0,M,сотрудник,0,135823,образование,C
8,2,6929,35,0,1,F,сотрудник,0,95856,свадьба,C
9,0,2188,41,1,0,M,сотрудник,0,144425,недвижимость,C


In [66]:
#для наглядности выведем таблицу с категориями доходов
columns = ['purpose_category', 'purpose']
data = [['E','0-30000'], ['D','30001-50000'], ['C','50001-20000'], ['B','200001-1000000'], ['A','свыше 1 млн']]
purpose_category_table = pd.DataFrame(data=data, columns=columns)
purpose_category_table

,purpose_category,purpose
0,E,0-30000
1,D,30001-50000
2,C,50001-20000
3,B,200001-1000000
4,A,свыше 1 млн


## Шаг 5. Категоризация целей кредита.

Так как в данных столбца **purpose** имеется достаточное количество дубликатов, также проведем их категоризацию, чтобы легче воспринимать информацию о цели кредита.  

In [67]:
# функция для категоризации целей кредита
def purpose_category_group(purpose):
    if  'авто' in purpose:
        return 'операции с автомобилем'
    elif 'недвиж' in purpose or 'жиль' in purpose:
        return 'операции с недвижимостью'
    elif 'свадьб' in purpose:
        return 'проведение свадьбы'
    elif 'образо' in purpose:
        return 'получение образования'

df['purpose_category'] = df['purpose'].apply(purpose_category_group)
df.head(15)

,children,days_employed,dob_years,education_id,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category
0,1,8437,42,0,0,F,сотрудник,0,253875,недвижимость,B,операции с недвижимостью
1,1,4024,36,1,0,F,сотрудник,0,112080,автомобиль,C,операции с автомобилем
2,0,5623,33,1,0,M,сотрудник,0,145885,недвижимость,C,операции с недвижимостью
3,3,4124,32,1,0,M,сотрудник,0,267628,образование,B,получение образования
4,0,14177,53,1,1,F,пенсионер,0,158616,свадьба,C,проведение свадьбы
5,0,926,27,0,1,M,компаньон,0,255763,недвижимость,B,операции с недвижимостью
6,0,2879,43,0,0,F,компаньон,0,240525,недвижимость,B,операции с недвижимостью
7,0,152,50,1,0,M,сотрудник,0,135823,образование,C,получение образования
8,2,6929,35,0,1,F,сотрудник,0,95856,свадьба,C,проведение свадьбы
9,0,2188,41,1,0,M,сотрудник,0,144425,недвижимость,C,операции с недвижимостью


### 1. Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [68]:
df_children_pivot = df.pivot_table(index=['gender'], columns='children', values='debt', aggfunc='sum')
df_children_pivot

children,0,1,2,3,4,5
gender,,,,,,
F,589,243,134,17,1,0
M,469,198,60,10,3,0


**Вывод 1**

Зависимость между количеством детей и возвратом кредита в срок есть. По сводной таблице видно, что с увеличением количества детей количество просрочек снижается, а те, кто имеет 5 детей, прросрочек вообще не имеют.

Но! Все же лучше посмотреть не количество просрочек, а их долю, так как количество еще не отражает всей "картины". 

In [69]:
debt_children = pd.DataFrame()
debt_children['count_children'] = df.groupby('children')['debt'].count()
debt_children['sum_children'] = df.groupby('children')['debt'].sum()
debt_children['result_children, %'] = ((debt_children['sum_children'] / debt_children['count_children']) * 100).round(2)
debt_children

,count_children,sum_children,"result_children, %"
children,,,
0,14021,1058,7.55
1,4792,441,9.20
2,2039,194,9.51
3,328,27,8.23
4,41,4,9.76
5,9,0,0.00


Результаты оказались такими: 
- У людей без детей доля просрочек наименьшая. Возможно это связано с тем, что бездетные люди имеют больше средств, так как они не обременены необходимостью ухода за ребенком/детьми.
- У людей с 1, 2 и 4 детьми достаточно высокая доля просрочек. Скорее всего это связано с тем, что с появлением детей больше денег требуется на содержание ребенка/детей, хотя государство оказывает поддержку при рождении ребенка, но похоже это не очень помогает. Возможно также, что 1 и 2 ребенка имеют молодые семьи, которые пока имеют недостаточный доход. 
- При этом у людей с 3 детьми меньшая доля просрочек. В нашей стране семья с 3 детьми считается многодетной и имеет дополнительную финансовую поддержку от государства, возможно по этой причине доля просрочек ниже. А вот при появлении 4 ребенка финансов даже при поддержке государства возможно не хватает.
- А вот 9 человек, имеющие 5 детей, всегда платили вовремя, но так как выборка людей с 5-ю детьми довольно небольшая, то будет немного голословно делать вывод, что *все* люди имеющие 5 детей возвращают кредит в срок, хоть таких людей в принципе не много.

**Вывод 2** 

Зависимость наличия задолженности от количества детей есть, но она не прямая. Тут играют роль социальные факторы, влияние которых довольно разнообразное и сложное, поэтому остановимся на выводе, что у людей, не имеющих детей, более низкий процент просрочек, чем у людей с детьми. 

### 2. Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [70]:
df_family_status = df.pivot_table(index='family_status_id', columns='gender', values='debt', aggfunc='sum')
df_family_status

gender,F,M
family_status_id,,
0,523,400
1,231,152
2,51,11
3,61,23
4,118,154


In [71]:
#для наглядности распечатаем таблицу с id семейного положения
columns = ['family_ststus_id', 'family_status']
data = [['0','женат / замужем '], ['1','гражданский брак'], ['2','Не женат / не замужем'], ['3','в разводе'], ['4','вдовец / вдова']]
purpose_category_table = pd.DataFrame(data=data, columns=columns)
purpose_category_table

,family_ststus_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,Не женат / не замужем
3,3,в разводе
4,4,вдовец / вдова


**Вывод 1**

По сводной таблице видим, что как мужчины, так и женщины в разводе или овдовевшие выплачивают кредит в срок чаще, чем не женатые или же состоящие в браке.

Но все же проверим долю возврата кредитов, так как просто "голое" количество не столь информативно (может быть количество заемщиков в разводе или оводовевших меньше, чем остальных).

In [72]:
debt_family_status = pd.DataFrame()
debt_family_status['sum_family_status'] = df.groupby('family_status_id')['debt'].sum()
debt_family_status['count_family_status'] = df.groupby('family_status_id')['debt'].count()
debt_family_status['result_family_status, %'] = ((debt_family_status['sum_family_status'] / debt_family_status['count_family_status']) * 100).round(2)
debt_family_status.sort_values('result_family_status, %', ascending = False)

,sum_family_status,count_family_status,"result_family_status, %"
family_status_id,,,
4,272,2780,9.78
1,383,4112,9.31
0,923,12213,7.56
3,84,1179,7.12
2,62,946,6.55


In [73]:
#для наглядности распечатаем таблицу с id семейного положения
columns = ['family_ststus_id', 'family_status']
data = [['0','женат / замужем '], ['1','гражданский брак'], ['2','Не женат / не замужем'], ['3','в разводе'], ['4','вдовец / вдова']]
purpose_category_table = pd.DataFrame(data=data, columns=columns)
purpose_category_table

,family_ststus_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,Не женат / не замужем
3,3,в разводе
4,4,вдовец / вдова


**Вывод 2**

Теперь же видно, что доля просрочек в принципе соизмерима (от 6,55% до 9,77%), но наибольшая доля просрочек наблюдается у людей не в браке и у людей, состоящих в гражденском браке. В предыдущей сводной таблице на первом месте по количеству просрочек были женатые и замужние, а сейчас они на 3 месте по общему % задолженности. 
Наименьший процент просрочки по возврату кредита имеют вдовцы/вдовы.

Таким образом, зависимость между семейным положением и возвратом кредита в срок наблюдается.

### 3. Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [74]:
df_total_income = df.pivot_table(index='total_income_category', columns='gender', values='debt', aggfunc='sum')
df_total_income 

gender,F,M
total_income_category,,
A,0,2
B,162,191
C,805,541
D,15,6
E,2,0


**Вывод 1**

Люди с доходом от 50 тыс. до 1 млн. имеют большее количество задолженностей, чем люди с малым доходом или с доходом, превышающим 1 млн.

Но проверим долю тех, кто не платит в срок, от общего количества по уровню дохода.

In [75]:
debt_total_income = pd.DataFrame()
debt_total_income['sum_total_income_category'] = df.groupby('total_income_category')['debt'].sum()
debt_total_income['count_total_income_category'] = df.groupby('total_income_category')['debt'].count()
debt_total_income['result_total_income_category, %'] = ((debt_total_income['sum_total_income_category'] / debt_total_income['count_total_income_category']) * 100).round(2)
debt_total_income.sort_values('result_total_income_category, %', ascending = False)

,sum_total_income_category,count_total_income_category,"result_total_income_category, %"
total_income_category,,,
E,2,22,9.09
C,1346,15850,8.49
A,2,25,8.00
B,353,4986,7.08
D,21,347,6.05


In [76]:
#для наглядности распечатаем таблицу с категориями доходов
columns = ['purpose_category', 'purpose']
data = [['E','0-30000'], ['D','30001-50000'], ['C','50001-20000'], ['B','200001-1000000'], ['A','свыше 1 млн']]
purpose_category_table = pd.DataFrame(data=data, columns=columns)
purpose_category_table

,purpose_category,purpose
0,E,0-30000
1,D,30001-50000
2,C,50001-20000
3,B,200001-1000000
4,A,свыше 1 млн


**Вывод 2**

Теперь получаем наибольший процент просрочки у людей с меньшим доходом (менее 30 тыс.), что логично, так как людям с небольшим доходом сложно обеспечивать свою жизнь (или жизнь всей семьи) и при этом платить вовремя кредит. 
Наименьший процент просрочек у людей с доходами от 30 тыс. до 50 тыс., хотя казалось бы наименьший процент должен быть у людей с высокими доходами, но, возможно, люди с доходами 30-50 тыс. берут кредит на меньшие суммы (но это не точно).

### 4. Как разные цели кредита влияют на его возврат в срок?

In [77]:
df_purpose_category = df.pivot_table(index='purpose_category', columns='gender', values='debt', aggfunc='sum')
df_purpose_category

gender,F,M
purpose_category,,
операции с автомобилем,229,168
операции с недвижимостью,443,334
получение образования,206,163
проведение свадьбы,106,75


**Вывод 1**

Больше всего просрочек при операциях с недвижимостью, что имеет свою логику, так как недвижимость имеет более высокий ценник, и, сооответственно, размер кредита в этой категории должен быть выше, поэтому погашать в срок достаточно большие суммы сложнее.

Меньше просрочек в случае кредитов на свадьбу, так как свадьбы менее затратны (хоть и свадьбы бывают очень помпезными, но вряд-ли те, кто могут себе позволить шикарную дорогую свадьбу, будут брать кредит на это мероприятие).

Но, опять же, для наглядности проверим долю тех, кто позволял себе просрочку погашения кредита, в каждой категории цели кредита.

In [78]:
debt_purpose_category = pd.DataFrame()
debt_purpose_category['sum_purpose_category'] = df.groupby('purpose_category')['debt'].sum()
debt_purpose_category['count_purpose_category'] = df.groupby('purpose_category')['debt'].count()
debt_purpose_category['result_purpose_category, %'] = ((debt_purpose_category['sum_purpose_category'] / debt_purpose_category['count_purpose_category']) * 100).round(2)
debt_purpose_category.sort_values('result_purpose_category, %', ascending = False)

,sum_purpose_category,count_purpose_category,"result_purpose_category, %"
purpose_category,,,
операции с автомобилем,397,4258,9.32
получение образования,369,3970,9.29
проведение свадьбы,181,2299,7.87
операции с недвижимостью,777,10703,7.26


**Вывод 2**

Итак, полученные данных в процентах показало, что наибольший процент просрочек в категории "операции с автомобилем" и почти наравне с ней "получение образования", а категория "операции с недвижимостью" имеет наименьший процент просрочек, что говорит нам о неверном первом выводе, основанном на сводной таблице. 
Скорее всего высокий процент просрочек по кредитам на недвижимость связан со страхом остаться без жилья (без автомобиля жить можно, а без дома остаться не хочет никто), а высокий процент по кредитам на образование может быить связан с тем, что учиться и работать одновременно, чтобы отдавать кредит за обучение - достаточно сложно. 

## Общий вывод:

Проанализировав входные данные можно сделать вывод, что семейное положение и количество детей влияет на возврат кредита в срок: 
- клиенты, не состоящие в браке или состоящие в гражданском браке имеют большую долю просрочек по кредитам;
- клиенты, не имеющие детей, или имеющие 3 или 5 детей чаще погашают кредит в срок, чем люди, имеющие 1, 2 или 4 ребенка;

Также на возврат кредита в срок влияет цель оформления кредита и уровень доходов:
- клиенты, которые оформляют кредит на операции с автомобилем или на образование, имеют более высокий процент просрочек, чем клиенты, целью оформления кредита которых является проведение свадьбы или операции с недвижимостью;
- клиенты, имеющие доход ниже 30 тыс. чаще имеют проблемы с погашением кредита, чем люди с более высокими доходами, но при этом клиенты, чей доход составляет от 30 до 50 тыс. чаще погашают кредит в срок. 